In [1]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import pandas as pd

In [ ]:
email_data = pd.read_csv("http://www.minethatdata.com/Kevin_Hillstrom_MineThatData_E-MailAnalytics_DataMiningChallenge_2008.03.20.csv")
email_data.head(3)

In [ ]:
male_df = email_data[email_data["segment"] != "Womens E-Mail"]
male_df["treatment"] = male_df["segment"].map(lambda x: 1 if x == "Mens E-Mail" else 0)
male_df.head(3)

In [ ]:
treatment_data = male_df[male_df["treatment"] == 1]
control_data = male_df[male_df["treatment"] == 0]

treatment_biased = treatment_data.drop(treatment_data[(
    (treatment_data['history'] > 300) |
    (treatment_data['recency'] < 6) |
    (treatment_data['recency'] == 'Multichannel')
) == False].sample(frac=0.5, random_state=1).index)

control_biased = control_data.drop(control_data[
    (control_data['history'] > 300) |
    (control_data['recency'] < 6) |
    (control_data['recency'] == 'Multichannel')
].sample(frac=0.5, random_state=1).index)

biased_data = pd.concat([treatment_biased, control_biased], axis=0)
biased_data.head(3)

In [ ]:
import statsmodels.formula.api as smf

In [ ]:
biased_reg = smf.ols(
    data=
)